In [3]:
import sys
sys.path.append('./python-classifier-2022/utils')
from data_reader import train_loader

In [ ]:
import sys
sys.path.append('./python-classifier-2022')
from LCNN_model import Wav2Vec2_LCNN as wav2vec2_lcnn
from CNN_outcome import CNN as cnn_outcome
import sys
sys.path.append('./python-classifier-2022/utils')
from data_reader import train_loader
import time
import torch
from torch.utils.data import Dataset,DataLoader
from logger import get_logger
from sklearn.metrics import f1_score

def train_challenge_model(data_folder, model_folder, verbose):
    #gpu setting
    torch.cuda.empty_cache()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    torch.cuda.set_device(device)
    
    
    # Find data files.
    if verbose >= 1:
        print('Finding data files...')

    # Find the patient data files.
    patient_files = find_patient_files(data_folder)  
    num_patient_files = len(patient_files)

    if num_patient_files==0:
        raise Exception('No data was provided.')

    # Create a folder for the model if it does not already exist.
    os.makedirs(model_folder, exist_ok=True)
    #log
    os.makedirs('./log/', exist_ok=True)
    
    
    logger = get_logger('./log/'  + 'model_log')

    

    # Extract the features and labels.
    if verbose >= 1:
        print('Extracting features and labels from the Challenge data...')

    murmur_classes = ['Present','Unknown', 'Absent']  
    num_murmur_classes = len(murmur_classes)
    outcome_classes = ['Abnormal', 'Normal']
    num_outcome_classes = len(outcome_classes)
    
    
    # Define the model /murmur
    wav2vec2_lcnn_murmur = wav2vec2_lcnn("murmur")
    wav2vec2_lcnn_murmur = wav2vec2_lcnn_murmur.cuda(device)
    
    # Define the model /outcome
    cnn_outcome_model = cnn_outcome()
    cnn_outcome_model = cnn_outcome_model.cuda(device)
    
    
    # optimizer, loss
    opt = torch.optim.Adam(wav2vec2_lcnn_model.parameters(), lr=1e-6, betas=(0.9, 0.999))
    scheduler = torch.optim.lr_scheduler.ExponentialLR(opt, 0.96)
    criterion = torch.nn.BCELoss()
    
    
    #
    
    #data loader
    trainset = train_loader(data_folder)
#     validset = train_loader(args.valid_data_folder )


    dataloader_train = DataLoader(dataset=trainset,
                    batch_size=6,
                           shuffle =True,
                           num_workers=0)
#     dataloader_valid = DataLoader(dataset=validset,
#                     batch_size=args.minibatchsize_valid,
#                            num_workers=args.dev_num_workers)
    

    # Train the model.
    all_file_train = len(trainset)
    dataloader_train_len = len(dataloader_train)
    for iter_ in range(20):
        train_pred_label,  train_gt_label = [], []
        start_time = time.time()
        running_loss = 0.0
        idx = 0
        wav2vec2_lcnn_murmur.train()
        for feature, _, _, _, _, _, _, current_murmur, _  in tqdm(dataloader_train): 
            feature = feature.cuda()
            feature = torch.tensor(feature, dtype=torch.float32)
    #         current_age_group = current_age_group.cuda()
    #         sex_feature = sex_feature.cuda()
    #         height_weight = height_weight.cuda()
    #         preg_feature = preg_feature.cuda()
    #         loc_feature = loc_feature.cuda()
            current_murmur = current_murmur.cuda()
    #         current_outcome = current_outcome.cuda()






            output_train = wav2vec2_lcnn_murmur(feature)   # output
            output_train = output_train.squeeze(1)


            output_preds_labels = (torch.ceil(output_train-0.4)).data.cpu().numpy()
            train_pred_label.extend(list(output_preds_labels))
            train_gt_label.extend(list(current_murmur.data.cpu().numpy()))

    #         murmur_weight = [4 if i==1 else 1 for i in list(current_murmur.data.cpu().numpy())]

    #         criterion = torch.nn.BCELoss(weight = torch.tensor(murmur_weight).cuda())
            loss = criterion(output_train.to(torch.float32), current_murmur.to(torch.float32))




            idx +=1              

            opt.zero_grad()
            loss.backward()
            opt.step()

            running_loss += loss.item()

        train_f1 = f1_score(y_true= train_gt_label, y_pred= train_pred_label)


        scheduler.step()
        print("lr: ", opt.param_groups[0]['lr'])
        logger.info("Iteration:{0}, train loss = {1:.6f} ,train F1 = {2:.6f} ".format(iter_, 
                     running_loss/dataloader_train_len,train_f1))


    # Save the model.
    save_challenge_model(model_folder, wav2vec2_lcnn_murmur)
    
    
    
    all_file_train = len(trainset)
    dataloader_train_len = len(dataloader_train)
    output_preds_labels_train = torch.empty((all_file_train,))
    output_preds_probs_train = torch.empty((all_file_train,2))
    output_gt_labels_train = torch.empty((all_file_train,))
    for iter_ in range(25):
        start_time = time.time()
        running_loss = 0.0
        running_f1 = 0.0
        running_acc = 0.0
        idx = 0
        cnn_outcome_model.train()
        for _, log_mel_feature, current_age_group, sex_feature, height_weight, preg_feature, loc_feature, _, current_outcome in tqdm.tqdm(dataloader_train):
            log_mel_feature = log_mel_feature.cuda()
            current_age_group = current_age_group.cuda()
            sex_feature = sex_feature.cuda()
            height_weight = height_weight.cuda()
            preg_feature = preg_feature.cuda()
            loc_feature = loc_feature.cuda()
#             current_murmur = current_murmur.cuda()
            current_outcome = current_outcome.cuda()




            output_train = cnn_outcome_model(log_mel_feature,current_age_group,sex_feature,height_weight, preg_feature,loc_feature)
            loss = criterion(F.softmax(output_train), current_outcome)



            idx +=1              

            opt.zero_grad()
            loss.backward()
            opt.step()

            running_loss += loss.item()


        scheduler.step()
        print("lr: ", opt.param_groups[0]['lr'])
        logger.info("Iteration:{0}, train loss = {1:.6f}".format(iter_, 
                     running_loss/dataloader_train_len)) #,train F1 = {2:.6f} ,train ACC = {3:.6f} /,train_f1,train_acc
    
    # Save the model.
    save_challenge_model(model_folder, cnn_outcome_model)
    
    

    if verbose >= 1:
        print('Done.')

# Load your trained model. This function is *required*. You should edit this function to add your code, but do *not* change the
# arguments of this function.
def load_challenge_model(model_folder, verbose):
    filename = os.path.join(model_folder, 'model.sav')
    return joblib.load(filename)

# Run your trained model. This function is *required*. You should edit this function to add your code, but do *not* change the
# arguments of this function.
def run_challenge_model(model, data, recordings, verbose):
    imputer = model['imputer']
    murmur_classes = model['murmur_classes']
    murmur_classifier = model['murmur_classifier']
    outcome_classes = model['outcome_classes']
    outcome_classifier = model['outcome_classifier']

    # Load features.
    features = get_features(data, recordings)

    # Impute missing data.
    features = features.reshape(1, -1)
    features = imputer.transform(features)

    # Get classifier probabilities.
    murmur_probabilities = murmur_classifier.predict_proba(features)
    murmur_probabilities = np.asarray(murmur_probabilities, dtype=np.float32)[:, 0, 1]
    outcome_probabilities = outcome_classifier.predict_proba(features)
    outcome_probabilities = np.asarray(outcome_probabilities, dtype=np.float32)[:, 0, 1]

    # Choose label with highest probability.
    murmur_labels = np.zeros(len(murmur_classes), dtype=np.int_)
    idx = np.argmax(murmur_probabilities)
    murmur_labels[idx] = 1
    outcome_labels = np.zeros(len(outcome_classes), dtype=np.int_)
    idx = np.argmax(outcome_probabilities)
    outcome_labels[idx] = 1

    # Concatenate classes, labels, and probabilities.
    classes = murmur_classes + outcome_classes
    labels = np.concatenate((murmur_labels, outcome_labels))
    probabilities = np.concatenate((murmur_probabilities, outcome_probabilities))

    return classes, labels, probabilities


# Save your trained model.
def save_challenge_model(model_folder, model):
    torch.save(model.state_dict(), os.path.join(model_folder, "{}.model".format(model)))

In [ ]:
    torch.save(wav2vec2_lcnn_model.state_dict(), os.path.join(model_folder, "{}_{}.model".format(args.model_name, iter_)))

    end_time = time.time()
    logger.info("Time used for each epoch training: {} seconds.".format(end_time - start_time))

In [ ]:
# Save your trained model.
def save_challenge_model(model_folder, model):
    torch.save(model.state_dict(), os.path.join(model_folder, "{}.model".format(model)))
# Extract features from the data.
def get_features(data, recordings):
    # Extract the age group and replace with the (approximate) number of months for the middle of the age group.
    age_group = get_age(data)

    if compare_strings(age_group, 'Neonate'):
        age = 0.5
    elif compare_strings(age_group, 'Infant'):
        age = 6
    elif compare_strings(age_group, 'Child'):
        age = 6 * 12
    elif compare_strings(age_group, 'Adolescent'):
        age = 15 * 12
    elif compare_strings(age_group, 'Young Adult'):
        age = 20 * 12
    else:
        age = float('nan')

    # Extract sex. Use one-hot encoding.
    sex = get_sex(data)

    sex_features = np.zeros(2, dtype=int)
    if compare_strings(sex, 'Female'):
        sex_features[0] = 1
    elif compare_strings(sex, 'Male'):
        sex_features[1] = 1

    # Extract height and weight.
    height = get_height(data)
    weight = get_weight(data)

    # Extract pregnancy status.
    is_pregnant = get_pregnancy_status(data)

    # Extract recording locations and data. Identify when a location is present, and compute the mean, variance, and skewness of
    # each recording. If there are multiple recordings for one location, then extract features from the last recording.
    locations = get_locations(data)

    recording_locations = ['AV', 'MV', 'PV', 'TV', 'PhC']
    num_recording_locations = len(recording_locations)
    recording_features = np.zeros((num_recording_locations, 4), dtype=float)
    num_locations = len(locations)
    num_recordings = len(recordings)
    if num_locations==num_recordings:
        for i in range(num_locations):
            for j in range(num_recording_locations):
                if compare_strings(locations[i], recording_locations[j]) and np.size(recordings[i])>0:
                    recording_features[j, 0] = 1
                    recording_features[j, 1] = np.mean(recordings[i])
                    recording_features[j, 2] = np.var(recordings[i])
                    recording_features[j, 3] = sp.stats.skew(recordings[i])
    recording_features = recording_features.flatten()

    features = np.hstack(([age], sex_features, [height], [weight], [is_pregnant], recording_features))

    return np.asarray(features, dtype=np.float32)
